In [ ]:
OPEN_API_KEY=''
import nest_asyncio
nest_asyncio.apply()

#design simple tool
from llama_index.core.tools import FunctionTool

def subtract(a:int,b:int)->int:
    "Subtract two numbers"
    return a-b
def mysteryFunction(a:int,b:int)->int:
    "mystery function operates on top of two numbers"
    return (a+b)*(a+b)

subtract_tool=FunctionTool.from_defaults(fn=subtract)
mystery_tool=FunctionTool.from_defaults(fn=mysteryFunction)

from llama_index.llms.openai import OpenAI
llm=OpenAI(model="gpt-turbo-3.5")
response=llm.predict_and_call([subtract_tool,mystery_tool],"Answer for the mystery function 10 and 5",verbose=True)
print(str(response))

#load data
from llama_index.core import SimpleDirectoryReader
documents=SimpleDirectoryReader(input_files="/files/metagpt.pdf").load_data()

from llama_index.core.node_parser import SentenceSplitter
splitter=SentenceSplitter(chunk_size=1024)
node_parser=splitter.get_nodes_from_documents(documents)

print(node_parser[0].get_content(metadata_mode=all))

from llama_index.core import VectorStoreIndex

vectorIndex=VectorStoreIndex(node_parser)
query_engine=vectorIndex.as_query_engine(similarity_top_k=2)

from llama_index.core.vector_stores import MetadataFilters
query_engine=vectorIndex.as_query_engine(similarity_top_k=2,
                                         filters=MetadataFilters.from_dicts([{"key":"page_label","value":"2"}]))

response=query_engine.query("Summarize the content of page 1")

print(str(response))

#define auto retrival
from typing import List
from llama_index.core.vector_stores import FilterCondition

def vector_query(query:str,page_number:List[str])->str:
    """Perform vector search ove an index
    query(str): the string query to be embedded.
    page_numbers:(list[str])Filter set for pages"""

    meta_dicts=[{"key":"page_label","value":p}for p in page_number]
    query_engine=vectorIndex.as_query_engine(similarity_top_k=2,
                                             filters=MetadataFilters.from_dicts(meta_dicts),
                                             condition=FilterCondition.OR)   
    response=query_engine.query(query)
    return response
vector_query_tool=FunctionTool.from_defaults(name="vector_tool",fn=vector_query)

llm=OpenAI(model="gpt-turbo-3.5",temperature=0)
response=llm.predict_and_call([vector_query_tool],"What is results of metagpt as per page 3?",verbose=True)

for n in response.nodes:
    print(n.metadata)

from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool
summary_index=SummaryIndex(node_parser)
summary_query_engine=summary_index.as_query_engine(response_node="tree_summarize",use_async=True)
summary_tool=QueryEngineTool.from_defaults(name="summary_tool",query_engine=summary_query_engine,description="Use if need to summarize the content")

response=llm.predict_and_call([vector_query_tool,summary_tool],"What are comparisons of metagpt and  chatdev on page 8?",verbose=True)
for n in response.source_nodes:
    print(n.metadata)

response=llm.predict_and_call([vector_query_tool,summary_tool],"Summary of the paper?",verbose=True)

print(str(response))